In [33]:
import pandas as pd
!pip install pandas openpyxl


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
file_path1 = r"C:\Users\DelphineMertens\OneDrive - element61\Documents\Training - Hackaton\Ratical_Thinkers\Data\LineC_SessionData-individual rats_SampleData.xlsx"
df_nov = pd.read_excel(file_path1)

df_nov.head()

,SESSION_DATE,ID_EVALUATION_SESSION,ID_RAT,Name,Birthdate,Gender,WEIGHT,TEMPERATURE,ID_SAMPLE,ID_BL_DOTS,ID_BL_APOPO,STATUS_KNOWNPOS,STATUS_BLINDPOS,REWARD,HIT,SniffTime,SniffThreshold
0,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515562,1,1.0,False,False,False,False,0,0
1,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515546,1,1.0,False,False,False,False,0,0
2,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515422,1,NaN,False,False,False,False,0,0
3,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515559,1,1.0,False,False,False,False,0,0
4,2016-11-03,6924,98,Stephen,2014-07-21,M,NaN,30.5,515553,1,1.0,False,False,False,False,0,0


In [35]:
df_nov.columns.tolist()

['SESSION_DATE',
 'ID_EVALUATION_SESSION',
 'ID_RAT',
 'Name',
 'Birthdate',
 'Gender',
 'WEIGHT',
 'TEMPERATURE',
 'ID_SAMPLE',
 'ID_BL_DOTS',
 'ID_BL_APOPO',
 'STATUS_KNOWNPOS',
 'STATUS_BLINDPOS',
 'REWARD',
 'HIT',
 'SniffTime',
 'SniffThreshold']

In [69]:
# Select and rename only the columns you want
df_nov_selected = df_nov.rename(columns={
    'Name': 'Rat_Name',
    'ID_EVALUATION_SESSION': 'Evaluation_Session_ID',
    'Gender': 'Gender',
    'WEIGHT': 'Weight',
    'SniffTime': 'Sniff_Time',
    'SniffThreshold': 'Threshold_Time',
    'ID_BL_DOTS': 'Lab_Test'
}).copy()

# Calculate Age
df_nov_selected['Age'] = ((
    pd.to_datetime(df_nov['SESSION_DATE']) - pd.to_datetime(df_nov['Birthdate'])
).dt.days / 365.25).round(2)

df_nov_selected['Run'] = ''
df_nov_selected['Hole'] = ''
# Reorder to match your desired structure
final_columns = [
    'Rat_Name', 'Evaluation_Session_ID', 'Gender', 'Age', 'Weight',
    'Run', 'Hole', 'Sniff_Time', 'Threshold_Time', 'Lab_Test'
]
df_nov_final = df_nov_selected[final_columns]

In [70]:
df_nov_final.columns

Index(['Rat_Name', 'Evaluation_Session_ID', 'Gender', 'Age', 'Weight', 'Run',
       'Hole', 'Sniff_Time', 'Threshold_Time', 'Lab_Test'],
      dtype='object')

In [71]:
df_nov_final = df_nov_final.copy()

df_nov_final.loc[:, 'Lab_Result'] = df_nov_final['Lab_Test'].apply(lambda x: 1 if x == 1 else 0)

In [72]:
df_nov_final = df_nov_final.copy()
df_nov_final.loc[:, 'Result_Of_Rat'] = (
    df_nov_final['Sniff_Time'] >= df_nov_final['Threshold_Time']
).astype(int)

In [73]:
df_nov_final.columns

Index(['Rat_Name', 'Evaluation_Session_ID', 'Gender', 'Age', 'Weight', 'Run',
       'Hole', 'Sniff_Time', 'Threshold_Time', 'Lab_Test', 'Lab_Result',
       'Result_Of_Rat'],
      dtype='object')

In [74]:
df_nov_final = df_nov_final.copy()

# FP: Rat says Positive (1), Lab says Negative (0)
df_nov_final['FP'] = ((df_nov_final['Result_Of_Rat'] == 1) & (df_nov_final['Lab_Result'] == 1)).astype(int)

# FN: Rat says Negative (0), Lab says Positive (1)
df_nov_final['FN'] = ((df_nov_final['Result_Of_Rat'] == 0) & (df_nov_final['Lab_Result'] == 1)).astype(int)

# TP: Rat says Positive (1), Lab says Positive (1)
df_nov_final['TP'] = ((df_nov_final['Result_Of_Rat'] == 1) & (df_nov_final['Lab_Result'] == 1)).astype(int)

# TN: Rat says Negative (0), Lab says Negative (0)
df_nov_final['TN'] = ((df_nov_final['Result_Of_Rat'] == 0) & (df_nov_final['Lab_Result'] == 0)).astype(int)

In [75]:
df_nov_final.head()

,Rat_Name,Evaluation_Session_ID,Gender,Age,Weight,Run,Hole,Sniff_Time,Threshold_Time,Lab_Test,Lab_Result,Result_Of_Rat,FP,FN,TP,TN
0,Stephen,6924,M,2.29,NaN,,,0,0,1,1,1,1,0,1,0
1,Stephen,6924,M,2.29,NaN,,,0,0,1,1,1,1,0,1,0
2,Stephen,6924,M,2.29,NaN,,,0,0,1,1,1,1,0,1,0
3,Stephen,6924,M,2.29,NaN,,,0,0,1,1,1,1,0,1,0
4,Stephen,6924,M,2.29,NaN,,,0,0,1,1,1,1,0,1,0
